# Gym Workout

TODO

NOW
- Try the simann approach

LATER
- Penalize the env for long-term differences

DONE
- Make the agent training stochastic

Imports

In [1]:
import gym
import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)
import random
import datetime
import itertools
import math

import matplotlib
#matplotlib.use("Pdf")
import matplotlib.pyplot as plt
%matplotlib inline

Create the session. An interactive session is a session that is automatically your default session.

In [2]:
sess = tf.InteractiveSession()

# My own little library of helper functions!

In [3]:
def fc(input_tensor, in_size, out_size, collection=None, non_linearity=tf.nn.sigmoid):
    
    collections = [tf.GraphKeys.VARIABLES, tf.GraphKeys.TRAINABLE_VARIABLES]
    if collection is not None:
        collections.append(collection)
        
    W = tf.get_variable(
        name="W",
        initializer=tf.truncated_normal(
            shape=[in_size, out_size],
            mean=0.0,
            stddev=0.1
        ),
        collections=collections
    )
    
    b = tf.get_variable(
        name="b",
        initializer=tf.constant(
            value=0.1,
            shape=[out_size]
        ),
        collections=collections
    )
    
    return non_linearity( tf.matmul(input_tensor, W) + b )

# fc test
fc_test_template = tf.make_template("fc_test_template", fc, in_size=2, out_size=2)
op = fc_test_template([[0., 1.]])
tf.initialize_all_variables().run()
print op.eval()

def fc_stack(input_tensor, list_of_sizes, collection=None):
    result = input_tensor
    
    for layer_index in xrange(len(list_of_sizes)-1):

        with tf.variable_scope("layer"+str(layer_index)):
        
            if layer_index == len(list_of_sizes)-2:
                non_linearity = tf.identity
            else:
                non_linearity = tf.nn.relu
        
            result = fc(
                result,        
                in_size=list_of_sizes[layer_index],
                out_size=list_of_sizes[layer_index+1],
                collection=collection,
                non_linearity=non_linearity
            )
        
    return result

# fc_stack test
fc_stack_test_template = tf.make_template("fc_stack_test_template", fc_stack, list_of_sizes=[2, 3, 2])
op = fc_stack_test_template([[0., 1.]])
tf.initialize_all_variables().run()
print op.eval()

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def one_hot(index, name="one_hot"):
    return tf.one_hot(indices=index, depth=env.observation_space.n, on_value=1, off_value=0, axis=None, name=name).eval()

def getFeedDict(observation):
    if type(env.observation_space) == gym.spaces.discrete.Discrete:
        return {x:[one_hot(observation)]}
    elif type(env.observation_space) == gym.spaces.box.Box:
        return {x:[observation]}
    else:
        print "ERR"
        
class InitializeNewVariables:
    def __enter__(self):
        self.temp = set(tf.all_variables())
    def __exit__(self, type, value, traceback):
        sess.run(tf.initialize_variables(set(tf.all_variables()) - self.temp))

        
### OpenAI Gym specific stuff ###
def bound_to_env_action_space(my_action):
    my_action = tf.maximum(my_action, env.action_space.low)
    my_action = tf.minimum(my_action, env.action_space.high)
    return my_action
    
def bound_to_env_observation_space(my_observation):
    my_observation = tf.maximum(my_observation, env.observation_space.low)
    my_observation = tf.minimum(my_observation, env.observation_space.high)
    return my_observation

[[ 0.569  0.551]]
[[ 0.069  0.092]]


# Choose an environment

In [4]:
# env = gym.make('ConfirmationBiasEasy-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('TwoRoundNondeterministicReward-v0')
# env = gym.make('CartPole-v0')
env = gym.make("Pendulum-v0")

print(env.action_space)
ACTION_DIMS = env.action_space.shape[0]
print(env.observation_space)
OBSERVATION_DIMS = env.observation_space.shape[0]

[2016-06-09 16:22:51,504] Making new env: Pendulum-v0


Box(1,)
Box(3,)


Generate some random data with which to build our env model

In [5]:
old_observations = []
actions = []
observations = []
rewards = []
dones = []
initial_observations = []

# This is the actual simulator.

In [6]:
def generate_data(epsilon=0.0, num_episodes=2, episode_len=200, render=True, feed_dict={}):
    total_reward = 0
    
    for i_episode in range(num_episodes):
        observation = env.reset()
        observation = observation.astype(np.float32)
        initial_observations.append(observation)


        for timestep in xrange(episode_len):
            if render:
                env.render()

            if random.random() < epsilon:
                action = env.action_space.sample() #choose a random action
            else:
                feed_dict[agent_in] = np.reshape(observation, [1, 3])
                action = bound_to_env_action_space(agent_out).eval(feed_dict)[0]

            old_observation = observation

            observation, reward, done, info = env.step(action)
            total_reward += reward
            observation = observation.astype(np.float32)

            old_observations.append(old_observation)
            actions.append(action)
            observations.append(observation)
            rewards.append(reward)
            dones.append(done)

            if done:
                print("Episode finished after {} timesteps".format(timestep+1))
                break

    print "We now have " + str(len(observations)) + " observations in total."
    
    return total_reward / (num_episodes * episode_len)

In [26]:
generate_data(epsilon=1.0, num_episodes=100, render=False);

We now have 104664 observations in total.


# Here we make the env model

In [8]:
ENV_MODEL_OUT_DIMS = OBSERVATION_DIMS + 1 + 1
ENV_MODEL_IN_DIMS = OBSERVATION_DIMS + ACTION_DIMS

env_model = tf.make_template(
    "env_model",
    fc_stack,
    list_of_sizes=[ENV_MODEL_IN_DIMS, 1000, ENV_MODEL_OUT_DIMS],
    collection="env_model_vars"
)

env_model_predicted_observation = tf.make_template(
    'env_model_predicted_observation',
    lambda env_model_predicted_outs: tf.slice(
        env_model_predicted_outs,
        begin=[0, 0],
        size=[-1, OBSERVATION_DIMS],
        name="env_model_predicted_observation"
    )
)

env_model_predicted_reward = tf.make_template(
    'env_model_predicted_reward',
    lambda env_model_predicted_outs: tf.slice(
        env_model_predicted_outs,
        begin=[0, OBSERVATION_DIMS],
        size=[-1, 1],
        name="env_model_predicted_reward"
    )    
)

env_model_predicted_done = tf.make_template(
    'env_model_predicted_done',
    lambda env_model_predicted_outs: tf.slice(
        env_model_predicted_outs,
        begin=[0, OBSERVATION_DIMS + 1],
        size=[-1, 1],
        name="env_model_predicted_done"
    )
)

with tf.name_scope('env_model'):
    env_model_ins = tf.placeholder(dtype=tf.float32, shape=[None, ENV_MODEL_IN_DIMS], name='ins')
    env_model_outs = tf.placeholder(dtype=tf.float32, shape=[None, ENV_MODEL_OUT_DIMS], name='outs')
    env_model_predicted_outs = env_model(env_model_ins)
    print "env_model_predicted_outs:", env_model_predicted_outs

    env_model_loss = tf.reduce_mean(tf.square(tf.sub(env_model_predicted_outs, env_model_outs)), name="env_model_loss")
    print "env_model_loss:", env_model_loss

env_model_predicted_outs: Tensor("env_model/env_model/layer1/Identity:0", shape=(?, 5), dtype=float32)
env_model_loss: Tensor("env_model/env_model_loss:0", shape=(), dtype=float32)


# Make the agent

In [9]:
agent = tf.make_template(
    "agent",
    fc_stack,
    list_of_sizes=[3, 100, 1],
    collection="agent_vars"
)

agent_in = tf.placeholder(dtype=tf.float32, shape=[None, OBSERVATION_DIMS])
agent_out = agent(agent_in)

# Make giant stack of agent, env, agent, env, etc.

In [10]:
intermediate_env_rewards = []
intermediate_actions = []
intermediate_observations = []

with InitializeNewVariables():
    prob_not_done = tf.Variable(tf.constant(1.), trainable=False, name="prob_not_done")
    total_predicted_reward = tf.Variable(tf.constant(0.), trainable=False, name="total_predicted_reward")

predicted_observations = tf.placeholder_with_default(
    np.asarray(initial_observations),
    [None, 3],
    name="initial_predicted_observations"
)

EPISODE_LENGTH = 200

for round in xrange(EPISODE_LENGTH):
    action = bound_to_env_action_space(agent(predicted_observations))
    
    predicted_outs = env_model(tf.concat(1, [predicted_observations, action]))
    
#     rewardFromThisRound = tf.mul(envModelPredictedReward(predictedOuts), probNotDone)
    reward_from_this_round = env_model_predicted_reward(predicted_outs)
    
    intermediate_env_rewards.append(reward_from_this_round)
    intermediate_actions.append(action)
    intermediate_observations.append(predicted_observations)
    
    total_predicted_reward = tf.add(total_predicted_reward, reward_from_this_round)
    
#     probEndingNow = tf.minimum(1, tf.maximum(0., envModelPredictedDone(predictedOuts)))
#     probNotDone *= 1 - probEndingNow
    
    predicted_observations = bound_to_env_observation_space(env_model_predicted_observation(predicted_outs))
    
agent_loss = -tf.reduce_mean(total_predicted_reward)
average_reward_per_action = tf.reduce_mean(tf.div(total_predicted_reward, EPISODE_LENGTH))

# Train the envModel and agent

In [11]:
tf.initialize_all_variables().run()

In [12]:
with InitializeNewVariables():
    env_model_optimizer = tf.train.AdamOptimizer(.01).minimize(env_model_loss, var_list=tf.get_collection("env_model_vars")) #BEST

In [13]:
with InitializeNewVariables():
    agent_optimizer = tf.train.AdamOptimizer(.1).minimize(agent_loss, var_list=tf.get_collection("agent_vars"))

In [14]:
#Run this after changing the agent's structure
tf.initialize_variables(tf.get_collection("agent_vars")).run()

In [21]:
#SimAnn optimizer like thing

collection_to_optimize = tf.get_collection("agent_vars")
# value_to_minimize = agent_loss

# current_value = value_to_minimize.eval()
current_value = generate_data(epsilon=0., num_episodes=1, episode_len=10, feed_dict=feed_dict, render=False)
print "current_value:", current_value

for iteration in xrange(5):
    feed_dict = {}

    for tensor in collection_to_optimize:
        feed_dict[tensor] = tf.add(tensor, tf.random_normal(tf.shape(tensor))).eval()

#     new_value = value_to_minimize.eval(feed_dict=feed_dict)
    new_value = generate_data(epsilon=0., num_episodes=1, feed_dict=feed_dict, render=False)
    print "average_reward_per_action: ", average_reward_per_action.eval()
    
    if new_value < current_value:
        for var in feed_dict:
            var.assign(feed_dict[var]).eval()

        current_value = new_value
        print "average_reward_per_action: ", average_reward_per_action.eval()

We now have 24639 observations in total.
current_value: -2.14774266865


KeyboardInterrupt: 

In [27]:
print "Training on ", len(initial_observations), " episodes"

env_model_ins_data = np.concatenate((np.asarray(old_observations), np.asarray(actions)), 1)
print "env_model_ins_data.shape:", env_model_ins_data.shape

env_model_outs_data = np.concatenate((np.asarray(observations), np.asarray([rewards]).T, np.asarray([dones]).T), 1)
print "env_model_outs_data.shape:", env_model_outs_data.shape

print "Training the environment model on " + str(env_model_ins_data.shape[0]) + " observations"
print

AGENT_MINI_BATCH_SIZE = len(initial_observations) / 2
print "AGENT_MINI_BATCH_SIZE:", AGENT_MINI_BATCH_SIZE
ENV_MODEL_MINI_BATCH_SIZE = 1000
print "ENV_MODEL_MINI_BATCH_SIZE:", ENV_MODEL_MINI_BATCH_SIZE

for iteration in xrange(100):
    #Train the env:
    for i in xrange(3):
        mini_batch_indices = np.random.randint(env_model_ins_data.shape[0], size=ENV_MODEL_MINI_BATCH_SIZE)

        env_model_optimizer.run(feed_dict={
            env_model_ins: env_model_ins_data[mini_batch_indices, :],
            env_model_outs: env_model_outs_data[mini_batch_indices, :]
        })
    
    if iteration % 10 == 0:
        #Evaluate the env:
        print "new env_model_loss:", env_model_loss.eval(feed_dict={
            env_model_ins: env_model_ins_data,
            env_model_outs: env_model_outs_data
        })
    
        #Evaluate the agent:
        print "average_reward_per_action: ", average_reward_per_action.eval()
    
    #Train the agent:
    mini_batch_indices = np.random.randint(len(initial_observations), size=AGENT_MINI_BATCH_SIZE)
    
    agent_optimizer.run(feed_dict={
        predicted_observations: np.asarray(initial_observations)[mini_batch_indices, :]
    })

    if iteration % 10 == 0:
        #Evaluate the agent:
        print "average_reward_per_action: ", average_reward_per_action.eval()
    
print
print "average_reward_per_action: ", average_reward_per_action.eval()

Training on  529  episodes
env_model_ins_data.shape: (104664, 4)
env_model_outs_data.shape: (104664, 5)
Training the environment model on 104664 observations

AGENT_MINI_BATCH_SIZE: 264
ENV_MODEL_MINI_BATCH_SIZE: 1000
new env_model_loss: 0.0140952
average_reward_per_action:  -7.21167
average_reward_per_action:  -7.21166
new env_model_loss: 0.00919901
average_reward_per_action:  -7.36295
average_reward_per_action:  -7.36221
new env_model_loss: 0.00673637
average_reward_per_action:  -7.48454
average_reward_per_action:  -7.48451
new env_model_loss: 0.0053211
average_reward_per_action:  -7.67507
average_reward_per_action:  -7.67502
new env_model_loss: 0.00437061
average_reward_per_action:  -7.70916
average_reward_per_action:  -7.70134
new env_model_loss: 0.00381559
average_reward_per_action:  -7.73608
average_reward_per_action:  -7.72703
new env_model_loss: 0.00332808
average_reward_per_action:  -7.81378
average_reward_per_action:  -7.82057
new env_model_loss: 0.0030367
average_reward_per_

In [ ]:
generate_data(epsilon=0.0)

# Saver (not working)

# Code to run envs in parallel

# Test long-term errors in env_model

In [ ]:
state =  env.reset()
print "initial:", state

print

print "Real:", env.step(np.asarray([0]))

state = env_model(
    np.reshape(
        np.concatenate(
            (state.astype(np.float32), np.asarray([0.]).astype(np.float32)), 0
        ), (1, 4)
    )
).eval()
print "predicted:", state


for i in xrange(200):
    print
    real_state = env.step(np.asarray([0]))
    print "real_state:", real_state

    state = env_model(
        np.reshape(
            np.concatenate(
                (state[0][:3], np.asarray([0.]).astype(np.float32)), 0
            ), (1, 4)
        )
    ).eval()
    print "predicted:", state
    
    print "difference:", np.sum(real_state[0] - state[0][:3])

initial: [-0.738  0.675  0.994]

Real: (array([-0.786,  0.618,  1.5  ]), -5.8631032467148243, False, {})
predicted: [[-0.776  0.651  1.49  -5.829 -0.02 ]]

real_state: (array([-0.843,  0.538,  1.963]), -6.3549570814529046, False, {})
predicted: [[ -8.362e-01   5.667e-01   1.942e+00  -6.186e+00  -4.226e-03]]
difference: -0.0149590012109

real_state: (array([-0.901,  0.434,  2.366]), -7.0109559060898015, False, {})
predicted: [[ -9.011e-01   4.473e-01   2.334e+00  -6.836e+00   6.481e-03]]
difference: 0.0197284769727

real_state: (array([-0.951,  0.309,  2.692]), -7.8085079072573649, False, {})
predicted: [[ -9.581e-01   2.991e-01   2.641e+00  -7.733e+00   5.858e-03]]
difference: 0.0683815018172

real_state: (array([-0.986,  0.168,  2.924]), -8.7160739922585559, False, {})
predicted: [[ -1.010e+00   1.492e-01   2.847e+00  -8.797e+00  -7.470e-03]]
difference: 0.119728166439

real_state: (array([-1.   ,  0.016,  3.05 ]), -9.6943561071263034, False, {})
predicted: [[ -1.045e+00  -5.242e-03  